In [ ]:
#####TEST TFIDF - Cosine

In [13]:
import nltk
import string
from nltk.corpus import wordnet

import pickle
import random

from operator import itemgetter
from randomwordgenerator import randomwordgenerator

from scipy.spatial.distance import cosine
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
#nltk.download('punkt') #if needed

In [ ]:
# Usage for cosine similarity
#cosim = 1 - cosine(sentence_embedding1, sentence_embedding2)
#cosine_similarity(sentence_embedding1.reshape(1, -1), sentence_embedding2.reshape(1, -1))

In [5]:
# Load the dictionary holding mission statements
with open('all_Combined_Cleaned_Ratings_Selection.pickle', 'rb') as handle1:
    missionStatements = pickle.load(handle1)

In [6]:
# Extract mission statement and organization ID into a list to work with
missionStatementsList = []
for k,v in missionStatements.items():
    missionStatementsList.append([k, v[3]])

In [7]:
# Create stemmer, lower, remove punctuation, and cosine similarity function
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

# Get stem funciton
def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

# Get stems, remove punctuation, transform lower case
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1] #positions in the matrix for the similarity

In [125]:
# Create a copy of the mission statements dictionary and 
# shuffle to randomize which statements are analyzed
missionStatementsCopy = missionStatements.copy()

l = list(missionStatementsCopy.items())
random.shuffle(l)
missionStatementsCopyShuffled = dict(l)

## Below start the procedures for manipulating statements and testing whether 
## procedures (TFIDF + Cosine Similarity here) match the target mission statements

###In the first test, part of the mission statement is deleted (either half or 9/10th). And then the test is whether ###the original mission statement is matched with the altered mission statement. 

In [ ]:
#Test to see if the original mission statement can be matched by using 
#only part of a mission statement (either half or 9/10th)

# Collect results whether procedure matched the target statement or not
matchedList = []

for item in missionStatementsList:
    
    # Equals is just a temporary list to collect cosine similarity values 
    # and then order them to see which mission statement yielded the highest result
    equals=[]
    
    for key, value in missionStatementsCopyShuffled.items():
        
        # for half of the statement use this
        half_length = int(len(item[1])/2) 
        
        # for 9/10th of the statement use this
        # the variable name is left unchanged so that no further changes are needed
        #half_length = int(len(item[1])/10) 
        
        equals.append([cosine_sim(item[1][:half_length], value[3]), key])
        
           
    value_list_sorted = sorted(equals, key=itemgetter(0))

    first_Recom_key = value_list_sorted[-1][1]
    second_Recom_key = value_list_sorted[-2][1]
    third_Recom_key = value_list_sorted[-3][1]            
    

    # Collect the first (highest cosine value) of compared mission statement in matchedList for 
    # further processing and check if it matches with the target mission statement 
    # (i.e. manipulated vs actual statement) 
    # the second_Recom_key and third_Recom_key variables are not used here
    # optional print statement for progress
    if first_Recom_key == item[0]:
        matchedList.append("key matched correctly", item[0])
        print("key matched correctly", item[0])

    else:
        matchedList.append("different key", item[0])
        print("different key", item[0])

### Here, the first half of the mission statement is retained 
### and the second half is replaced with random words

In [ ]:
# First half is the actual mission statement 
# Second half of the mission statement is replaced with random words

# Collect results whether procedure matched the target statement or not
matchedList = []

for item in missionStatementsList:
    
    missionStatementAsList = item[1].split(' ')
    numberOfWordsInMissionStatement = len(missionStatementAsList)
    halfnumberOfWordsInMissionStatement = int(numberOfWordsInMissionStatement/2)
    
    firstHalf = missionStatementAsList[:halfnumberOfWordsInMissionStatement]
    secondHalf = randomwordgenerator.generate_random_words(n = halfnumberOfWordsInMissionStatement)
    
    constructedMissionList = firstHalf + secondHalf
    constructedMissionStatement = ''
    
    for i in constructedMissionList:
        constructedMissionStatement += i + " "
        
    
    # The equals list is just a temporary list to collect cosine similarity values 
    # And then order them to see which mission statement yielded the highest result
    equals=[]
    
    for key, value in missionStatements.items():
        
        equals.append([cosine_sim(constructedMissionStatement, value[3]), key])
        
    
    value_list_sorted = sorted(equals, key=itemgetter(0))

    first_Recom_key = value_list_sorted[-1][1]
    second_Recom_key = value_list_sorted[-2][1]
    third_Recom_key = value_list_sorted[-3][1]            

    
    # Collect the first (highest cosine value) of compared mission statement in matchedList for 
    # further processing and check if it matches with the target mission statement 
    # (i.e. manipulated vs actual statement) 
    # the second_Recom_key and third_Recom_key variables are not used here
    # optional print statement for progress
    if first_Recom_key == item[0]:
        matchedList.append("key matched correctly", item[0])
        print("key matched correctly", item[0])

    else:
        matchedList.append("different key", item[0])
        print("different key", item[0])

### Here 1/10th of the statement is retained and the remaining 9/10th are replaced with random words

In [ ]:
# Select mission statement and manipulate it. Retain 1/10th and replace 9/10th with random words
# and see if procedure returns the target mission statement

# Collect results whether procedure matched the target statement or not
matchedList = []

for item in missionStatementsList:
    
    missionStatementAsList = item[1].split(' ')
    numberOfWordsInMissionStatement = len(missionStatementAsList)
    halfnumberOfWordsInMissionStatement = int(numberOfWordsInMissionStatement/10)
    
    firstHalf = missionStatementAsList[:halfnumberOfWordsInMissionStatement]
    secondHalf = randomwordgenerator.generate_random_words(n = int(numberOfWordsInMissionStatement*(9/10)))
    
    constructedMissionList = firstHalf + secondHalf
    constructedMissionStatement = ''
    
    for i in constructedMissionList:
        constructedMissionStatement += i + " "
    
    equals=[]
    
    # Loop through dictionary and collect cosine similarity values
    for key, value in missionStatements.items():
        
        equals.append([cosine_sim(constructedMissionStatement, value[3]), key])
        
    
    # Sort list by value of cosine similarity
    value_list_sorted = sorted(equals, key=itemgetter(0))

    # Select the org ID with the highest cosine similarity value. Ranked 1 to 3
    first_Recom_key = value_list_sorted[-1][1]
    second_Recom_key = value_list_sorted[-2][1]
    third_Recom_key = value_list_sorted[-3][1]            

    
    
    # Append results to matchedList, plus optional print statements for progress
    if first_Recom_key == item[0]:
        matchedList.append(["key matched correctly", item[0]])
        print("key matched correctly", item[0])
    
    elif second_Recom_key == item[0]:
        matchedList.append(["key matched correctly on 2nd return", item[0]])
        print("key matched correctly on 2nd return", item[0])
    
    elif third_Recom_key == item[0]:
        matchedList.append(["key matched correctly on 3rd return", item[0]])
        print("key matched correctly on 3rd return", item[0])    
    
    else:
        matchedList.append(["different key", item[0]])
        print("different key", item[0])

### Here mission statement words are replaced with first synonym found

In [ ]:
# Substitute each word in the mission statement with the first synonym suggested by wordnet

matchedList = []

for item in missionStatementsList:
    
    missionStatementAsList = item[1].split(' ')
    
    constructedMissionList = []
    constructedMissionStatement = ''
    
    #synlist is just a temporary list used to collect the synonyms suggest by wordnet
    synlist = []
   
    for word in missionStatementAsList:

        synonyms = []

        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())
            
        a = set(synonyms)
        a = list(a)
        
        
        
        try:
            
            #just a check that the synonym is not equal the search word
            if a[0] != word:
                constructedMissionList.append(a[0])
            else:
                constructedMissionList.append(a[1])
                
        except:
            constructedMissionList.append("")
    

    for i in constructedMissionList:
        constructedMissionStatement += i + " "
        
    
    #equals is just a temporary list to collect cosine similarity values
    equals=[]
    
    for key, value in missionStatements.items():
        
        equals.append([cosine_sim(constructedMissionStatement, value[3]), key])
        
    
    value_list_sorted = sorted(equals, key=itemgetter(0))

    first_Recom_key = value_list_sorted[-1][1]
    second_Recom_key = value_list_sorted[-2][1]
    third_Recom_key = value_list_sorted[-3][1]            

    
    # Append results to matchedList, plus optional print statements for progress
    if first_Recom_key == item[0]:
        matchedList.append("key matched correctly", item[0])
        print("key matched correctly", item[0])
    
    elif second_Recom_key == item[0]:
        matchedList.append("key matched correctly on second", item[0])
        print("key matched correctly on second", item[0])
    
    elif third_Recom_key == item[0]:
        matchedList.append("key matched correctly on third", item[0])
        print("key matched correctly on third", item[0])
    
    else:
        matchedList.append("different key", item[0])
        print("different key", item[0])

### Here mission statement words are replaced with synonyms and only synonyms which are not found
### in the original mission statement. That is, all words are replaced with synonyms not seen before

In [ ]:
# Substitute words in statement with synonyms not in anywhere in statement

matchedList = []

for item in missionStatementsList:
    
    missionStatementAsList = item[1].split(' ')
    
    # Use these variables to create the new mission 
    # statement containing the synonyms 
    constructedMissionList = []
    constructedMissionStatement = ''
    
    synlist = []

    for word in missionStatementAsList:

        synonyms = []

        for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())
            
        a = set(synonyms)
        a = list(a)
        
        
        # Attempt to replace the word with the first three suggested synonyms by wordnet
        # If the first one equals the word to be replaced or already occurs in mission statement
        # then proceed to the second suggestion. If the first three suggestions do not meet the conditions
        # or if there are no suggestions (except clause), then simply leave blank replacement
        try:
                        
            if a[0] != word and a[0] not in missionStatementAsList:
                constructedMissionList.append(a[0])
            
            elif a[1] != word and a[1] not in missionStatementAsList:
                constructedMissionList.append(a[1])
                
            elif a[2] != word and a[2] not in missionStatementAsList:
                constructedMissionList.append(a[1])
            
            else:
                constructedMissionList.append("")
                
                
        except:
            constructedMissionList.append("")
    
     
    
    
    for i in constructedMissionList:
        constructedMissionStatement += i + " "
        
    
    # The equals lists collects the cosine values together with organization ID
    # to then sort the highest cosine value
    equals=[]
    
    for key, value in missionStatements.items():
        
        equals.append([cosine_sim(constructedMissionStatement, value[3]), key])
        
    
    value_list_sorted = sorted(equals, key=itemgetter(0))

    first_Recom_key = value_list_sorted[-1][1]
    second_Recom_key = value_list_sorted[-2][1]
    third_Recom_key = value_list_sorted[-3][1]            

    
    # Collect result in list for further processing. Option print statements for progress
    if first_Recom_key == item[0]:
        matchedList.append("key matched correctly", item[0])
        print("key matched correctly", item[0], loopduration)
    
    elif second_Recom_key == item[0]:
        matchedList.append("key matched correctly on 2nd return", item[0])
        print("key matched correctly on second", item[0], loopduration)
    
    elif third_Recom_key == item[0]:
        matchedList.append("key matched correctly on 3rd return", item[0])
        print("key matched correctly on third", item[0], loopduration)
    
    else:
        matchedList.append("different key", item[0])
        print("different key", item[0], loopduration)

In [3]:
import time

In [2]:
import pickle
# Load the dictionary holding mission statements
with open('all_Combined_Cleaned_Ratings_Selection.pickle', 'rb') as handle1:
    missionStatements = pickle.load(handle1)

In [7]:
for k, v in missionStatements.items():
    print(k, v[3])

13341 Girl Scouts builds girls of courage, confidence & character, who make the world a better place!
9731 Founded in 1952, the Arizona-Sonora Desert Museum is dedicated to the conservation of the Sonoran Desert. Its mission is to inspire people to live in harmony with the natural world by fostering love, appreciation, and understanding of the Sonoran Desert. The Arizona-Sonora Desert Museum is a world-renowned zoo, natural history museum and botanical garden, all in one place. Exhibits re-create the natural landscape of the Sonoran Desert Region so realistically you find yourself eye-to-eye with mountain lions, prairie dogs, Gila monsters, and more. Within the Museum grounds, you will see more than 300 animal species and 1,200 kinds of plants. There are almost two miles of paths traversing 21 acres of beautiful desert.
18563 The Children's Museum Tucson-Oro Valley's mission is to provide fun, play-based, interactive, hands-on learning experiences for children and their families. The C

9603 Atlanta Ronald McDonald House Charities' mission is to nurture the health and well-being of children and families. To advance our mission, we operate three programs: our core House Program, a Care Mobile Program and a Family Room Program.Our Houses are built on the simple idea that nothing else should matter when a family is focused on healing their child. Services include housing, meals, 24-hour transportation to and from the hospital, laundry facilities, positive distractions and connections with other families going through similar experiences. Our Ronald McDonald Care Mobile provides care for underserved children in metro Atlanta to address the number one diagnosis at Children's locations-asthma. With the Care Mobile, we aim to create stronger and healthier families. Our Family Room provides the comforts of home while in the hospital environment. We aim to provide a refuge for families just steps away from their child's hospital bed where they can rest, reflect and refuel.
924

13004 The Career Wardrobe's mission is to empower women in transition by providing programs that inspire the confidence necessary to achieve self-sufficiency and pursue a productive career. We do this by providing professional work attire, educational programs, and networking tools and opportunities. Career Wardrobe is the nation's largest independent, community-based nonprofit organization serving women transitioning into the workforce by providing professional attire. Since 1995, Career Wardrobe has assisted nearly 80,000 women by inspiring the confidence necessary to achieve independence.
17269 The mission of CareNet Pregnancy Centers, an outreach ministry of Jesus Christ, is equipping parents, teens and pre-teens to live lives of purpose, ministering to pregnant women and providing support to women and families through education and resources. CareNet was founded as a pro-life Christian organization and is committed to sharing the love of Christ in word and deed with everyone we ha

9712 The Center connects our creative community by providing a dedicated campus, supporting excellent programming, and nurturing a collaborative spirit.
17218 Community Coalition works to help transform the social and economic conditions in South LA that foster addiction, crime, violence and poverty by building a community institution that involves thousands in creating, influencing and changing public policy. For the past 25 years, Community Coalition has continued with this groundbreaking approach. We elevate the voices of our members, shift power to the community, and tackle the root causes of poverty, crime and violence. Together we are creating a more prosperous, safer and healthier South L.A.
12916 The mission of Community Cooperative is to eliminate hunger and homelessness in Lee County while simultaneously inspiring and supporting sustained positive change in its clients by delivering innovative food, education and social service programs. Community Cooperative strives to be th

8322 Council for a Strong America (CSA) is the umbrella nonprofit for five membership organizations comprising the unique and powerful voices of business, military, law enforcement, faith, and sports all working together to prepare young Americans for success. CSA serves as the corporate nonprofit for its membership organizations. Fight Crime: Invest in Kids was launched in 1996 followed by America's Edge (2008), Mission: Readiness (2009), Shepherding the Next Generation (2010) and Champions for America's Future (2013). Members of each CSA organization engage in a variety of activities including releasing research reports at press conferences, writing opinion pieces in the media and meeting with policy makers to educate them on effective investments that help children succeed.
13905 Council for Children's Rights is a legal advocacy program for children in the Southeast which stands up for every child's right to be safe, healthy, and educated. In 2006, the Council for Children and the C

...

9763 Girl Scouts of Western Washington (GSWW) serves more than 26,000 girls in grades K-12 in Western Washington. Girls participate in age-based programs as part of a troop or as individual Girl Scouts. GSWW offers girls camping and hands-on outdoor education opportunities at six camp properties located in Longview, Hoquiam, Belfair, Carnation and Lilliwaup on the Hood Canal. In addition to providing year-round program opportunities for girls, GSWW's camps are used by school groups and other youth-serving non-profits for environmental education and learning experiences. GSWW is the result of a merger between Girl Scouts of the Totem Council and Girl Scouts of the Pacific Peaks Council.
12252 Founded in 1912, Girl Scouts USA is the world's premiere leadership organization for girls. It provides an accepting, nurturing environment where girls can develop leadership skills, cultivate lifelong friendships, serve their communities and grow through new and exciting experiences. The area's fi

14662 Habitat for Humanity of Eagle and Lake Counties, founded in 1995, has since served 48 families by creating affordable homeownership opportunities. Habitat for Humanity in the Vail Valley is part of a global, nonprofit housing organization dedicated to eliminating substandard housing locally and worldwide through constructing, rehabilitating and preserving homes, by advocating for fair and just housing policies, and by providing training and access to resources to help families improve their shelter conditions.
15169 Habitat for Humanity of East Jefferson County seeks to put God's love into action, Habitat for Humanity of East Jefferson County brings people together to build homes, communities and hope. Habitat offers people a hand up rather than a handout. Community volunteers work side by side with neighbors in need to build and repair houses. Habitat homeowners pay back the cost of their housing materials through an affordable, no-profit mortgage. Their mortgage payments, toget

15750 The mission of the Illinois Holocaust Museum and Education Center is perhaps best expressed in our founding principle: Remember the Past, Transform the Future. The Museum is dedicated to preserving the legacy of the Holocaust by honoring the memories of those who were lost and by teaching universal lessons that combat hatred, prejudice and indifference. The Museum fulfills its mission through the exhibition, preservation and interpretation of its collections and through education programs and initiatives that foster the promotion of human rights and the elimination of genocide.
15231 The Illinois Network of Charter Schools advocates for the improvement of public education by leveraging the charter school model as a catalyst to transform lives and communities. As the voice of Illinois charter schools, INCS engages a diverse coalition of policymakers, school leaders, parents, and community members to create systemic change and secure high-quality schools for underserved communities

8874 To provide safe, loving homes and specialized services for children who have been abused, abandoned or neglected. 
17428 Jones Valley Teaching Farm's mission is encouraging students to act as critical thinkers, problem solvers, and change agents in their communities. We accomplish this mission by partnering with schools to provide students with innovative and rigorous hands-on food and nutrition education. We believe that our city's greatest resources are inside our schools and that education must connect discoveries in the classroom to action in the community.
3918 Founded by Joni Eareckson Tada in 1979, Joni and Friends communicates the gospel and equips Christ-honoring churches worldwide to evangelize and disciple people affected by disabilities. We present the clear and concise Gospel of Jesus Christ to all people with disabilities and their families served through our programs. We train, disciple, and mentor people affected by disability to exercise their gifts of leadership 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

